<a href="https://colab.research.google.com/github/icaroandri/Desafio-Alura-Store/blob/main/TELECOMX_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

url = 'https://raw.githubusercontent.com/ingridcristh/challenge2-data-science/refs/heads/main/TelecomX_Data.json'
df = pd.read_json(url)


customer_df = pd.json_normalize(df['customer'])
phone_df = pd.json_normalize(df['phone'])
internet_df = pd.json_normalize(df['internet'])
account_df = pd.json_normalize(df['account'])

df_flat = pd.concat([df[['customerID', 'Churn']],
                     customer_df, phone_df, internet_df, account_df], axis=1)

df_flat.head()

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4


In [13]:

print(df_flat.columns)


df_flat['Charges.Total'] = pd.to_numeric(df_flat['Charges.Total'], errors='coerce')
df_flat['Charges.Monthly'] = pd.to_numeric(df_flat['Charges.Monthly'], errors='coerce')


if 'customerID' in df_flat.columns:
    df_flat = df_flat.drop(columns=['customerID'])


print(df_flat.info())

Index(['customerID', 'Churn', 'gender', 'SeniorCitizen', 'Partner',
       'Dependents', 'tenure', 'PhoneService', 'MultipleLines',
       'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
       'PaperlessBilling', 'PaymentMethod', 'Charges.Monthly',
       'Charges.Total'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Churn             7267 non-null   object 
 1   gender            7267 non-null   object 
 2   SeniorCitizen     7267 non-null   int64  
 3   Partner           7267 non-null   object 
 4   Dependents        7267 non-null   object 
 5   tenure            7267 non-null   int64  
 6   PhoneService      7267 non-null   object 
 7   MultipleLines     7267 non-null   object 
 8   InternetService   7267 non-n

In [14]:

df_flat['Churn'] = df_flat['Churn'].map({'Yes':1, 'No':0})


df_encoded = pd.get_dummies(df_flat, drop_first=True)


print(df_encoded['Churn'].value_counts(normalize=True))


df_flat['Charges.Monthly'] = pd.to_numeric(df_flat['Charges.Monthly'], errors='coerce')
df_flat['Charges.Total'] = pd.to_numeric(df_flat['Charges.Total'], errors='coerce')

Churn
0.0    0.73463
1.0    0.26537
Name: proportion, dtype: float64


In [16]:

df_flat = df_flat.drop(columns=['customerID'], errors='ignore')

print("Formato final:", df_flat.shape)

Formato final: (7267, 20)


In [17]:
print("\nProporção de evasão (Churn):")
print(df_flat['Churn'].value_counts(normalize=True))


Proporção de evasão (Churn):
Churn
0.0    0.73463
1.0    0.26537
Name: proportion, dtype: float64


In [31]:

df_enc = pd.get_dummies(df_flat, drop_first=True)

X = df_enc.drop(columns=['Churn'])
y = df_enc['Churn']


scaler = StandardScaler()
X[X.select_dtypes(include=np.number).columns] = scaler.fit_transform(
    X.select_dtypes(include=np.number))

In [32]:
corr = df_enc.corr()['Churn'].sort_values(ascending=False)
print("\nVariáveis mais correlacionadas com Churn:")
print(corr.head(10))
print(corr.tail(10))


Variáveis mais correlacionadas com Churn:
Churn                             1.000000
InternetService_Fiber optic       0.308020
PaymentMethod_Electronic check    0.301919
Charges.Monthly                   0.193356
PaperlessBilling_Yes              0.191825
SeniorCitizen                     0.150889
StreamingTV_Yes                   0.063228
StreamingMovies_Yes               0.061382
MultipleLines_Yes                 0.040102
PhoneService_Yes                  0.011942
Name: Churn, dtype: float64
Charges.Total                          -0.199484
InternetService_No                     -0.227890
StreamingTV_No internet service        -0.227890
OnlineSecurity_No internet service     -0.227890
OnlineBackup_No internet service       -0.227890
DeviceProtection_No internet service   -0.227890
StreamingMovies_No internet service    -0.227890
TechSupport_No internet service        -0.227890
Contract_Two year                      -0.302253
tenure                                 -0.352229
Name: Chu

In [33]:

df = df.dropna(subset=['Churn'])

X = df.drop("Churn", axis=1)
y = df["Churn"]


from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

In [34]:

rf = RandomForestClassifier(n_estimators=200, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

print("\n==== Random Forest ====")
print(classification_report(y_test, y_pred_rf))
print("ROC AUC:", roc_auc_score(y_test, rf.predict_proba(X_test)[:,1]))
print("Matriz de confusão:\n", confusion_matrix(y_test, y_pred_rf))


ValueError: could not convert string to float: '3948-FVVRP'

In [30]:
importances = pd.Series(rf.feature_importances_, index=X.columns)
print("\nTop variáveis mais importantes (Random Forest):")
print(importances.sort_values(ascending=False).head(10))


AttributeError: 'RandomForestClassifier' object has no attribute 'estimators_'